In [ ]:
import sys
sys.path.insert(0,'../src/')

from data.make_dataset import *
from visualization.visualize import *
from models.models import *

%load_ext autoreload
%autoreload 1

In [ ]:
import torch
import numpy

if torch.cuda.is_available():
    prop = torch.cuda.get_device_properties(0)
    print(f"Using {prop.name} with {np.round(prop.total_memory/1e9, 2)}GB of RAM")
else:
    print("Using CPU")

# Load Data

In [ ]:
channel_size = 3
image_size = 64
batch_size = 128
workers = 10

In [ ]:
%%time
data_loader = make_dataloader("../data/raw/planctons_original", image_size, batch_size, workers)

## Visualize Data

In [ ]:
print_samples(data_loader)

# Load Models

In [ ]:
latent_size = 100
feature_map_size = 64

beta = 0.5
lr = 0.0002
num_epochs = 2

In [ ]:
g = DCGAN_Generator(latent_size, feature_map_size, channel_size)
d = DCGAN_Discriminator(feature_map_size, channel_size)

gan = GAN(d, g, lr, beta)

# Train Model

In [ ]:
%%time
gan.train(data_loader, num_epochs)

## Visualize training stats

In [ ]:
plot_loss(gan.G_losses, gan.D_losses)

In [ ]:
generator_progress(gan.img_list)

In [ ]:
compare_fake_real(data_loader, gan.device, gan.img_list)

## Use models

In [ ]:
fake_batch = gan.generate(torch.randn(64, gan.generator.latent_size, 1, 1, device=gan.device))
print_batch_images([fake_batch], gan.device)

In [ ]:
gan.predict_discriminator(next(iter(data_loader))[0])